In [1]:
from sqlalchemy import create_engine

%load_ext sql
engine = create_engine('duckdb:///:memory:', connect_args={'config': { 'allow_unsigned_extensions': 'true' }})
%sql engine

In [2]:
%%sql
INSTALL '../build/release/extension/erpel/erpel.duckdb_extension';
LOAD '../build/release/extension/erpel/erpel.duckdb_extension';
SELECT * from duckdb_extensions();

Running query in 'duckdb:///:memory:'

extension_name,loaded,installed,install_path,description,aliases
autocomplete,False,False,,Add supports for autocomplete in the shell,[]
erpel,True,None,/home/jr/.duckdb/extensions/v0.8.1/linux_amd64/erpel.duckdb_extension,,[]
fts,True,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[]
httpfs,False,True,/home/jr/.duckdb/extensions/v0.8.1/linux_amd64/httpfs.duckdb_extension,Adds support for reading and writing files over a HTTP(S) connection,"['http', 'https', 's3']"
icu,True,True,/home/jr/.duckdb/extensions/v0.8.1/linux_amd64/icu.duckdb_extension,Adds support for time zones and collations using the ICU library,[]
inet,False,False,,Adds support for IP-related data types and functions,[]
jemalloc,True,True,(BUILT-IN),Overwrites system allocator with JEMalloc,[]
json,True,True,/home/jr/.duckdb/extensions/v0.8.1/linux_amd64/json.duckdb_extension,Adds support for JSON operations,[]
motherduck,False,False,,Enables motherduck integration with the system,['md']
parquet,True,True,/home/jr/.duckdb/extensions/v0.8.1/linux_amd64/parquet.duckdb_extension,Adds support for reading and writing parquet files,[]


In [3]:
%%sql
SET sap_ashost TO 'vhcalnplci';
SET sap_sysnr='00';
SET sap_user='DEVELOPER';
SET sap_password='Bym38tpl';
SET sap_client='001';
SET sap_lang='EN';
SELECT * FROM duckdb_settings() where name like 'sap%' order by 1;

Running query in 'duckdb:///:memory:'

name,value,description,input_type
sap_ashost,vhcalnplci,The hostname SAP application server,VARCHAR
sap_client,001,Don't know yet,VARCHAR
sap_lang,EN,Language of the user to connect with,VARCHAR
sap_password,Bym38tpl,User password to login,VARCHAR
sap_sysnr,00,System number on the application server,VARCHAR
sap_user,DEVELOPER,Username to login,VARCHAR


In [4]:
%sql PRAGMA sap_rfc_ping

Running query in 'duckdb:///:memory:'

msg
PONG


In [5]:
%sql SELECT * FROM sap_rfc_search_function() LIMIT 10;

Running query in 'duckdb:///:memory:'

FUNCNAME,GROUPNAME,APPL,HOST,STEXT
/ASU/BP_JOBLOG_SHOW_SM37B_RFC,/ASU/SSM_SERVICES,,,RFC-Wrapper around BP_JOBLOG_SHOW_SM37B
/ASU/CHECK_UPGRADE_STATUS,/ASU/RFC_SERVICES,,,Determine the status of the ASU in upgrade use case
/ASU/DISPLAY_TEXT,/ASU/SERVICES,G,,Behandlung von Langtexten im Schedule-Manager
/ASU/OFFICE_DOCUMENT_SHOW,/ASU/OFFICE_INTEGRATION,K,,Schedule Manager: Office-Dokument bearbeiten
/ASU/RS_VARIANT_ADD,/ASU/SVAR,,,Hinzufuegen neue Variante
/ASU/RS_VARIANT_CATALOG,/ASU/SVAR,S,,Returns variant directory for a report in an internal table
/ASU/RS_VARIANT_CHANGE,/ASU/SVAR,,,Ändern Variante
/ASU/RS_VARIANT_DISPLAY,/ASU/SVAR,S,,Anzeigen der Variante
/ASU/RS_VARIANT_EXISTS,/ASU/SVAR,S,,Checks whether variant exists
/ASU/SCMA_TREAT_TEXT,/ASU/SERVICES,G,,Behandlung von Langtexten im Schedule-Manager


In [21]:
%sql SELECT * FROM sap_rfc_invoke('STFC_CONNECTION', {'REQUTEXT': 'Quack'});

Running query in 'duckdb:///:memory:'

ECHOTEXT,RESPTEXT
Quack,SAP R/3 Rel. 752 Sysid: NPL Date: 20230725 Time: 182316 Logon_Data: 001/DEVELOPER/E


In [9]:
%%sql
select ECHOSTRUCT.RFCFLOAT, ECHOSTRUCT.RFCCHAR1, ECHOSTRUCT.RFCDATE, ECHOSTRUCT.RFCTIME, ECHOSTRUCT.RFCINT4, ECHOSTRUCT.RFCDATA1 from sap_rfc_invoke('STFC_STRUCTURE', {
    'IMPORTSTRUCT': { 
        'RFCCHAR1': 'A', 
        'RFCCHAR2': 'BB',
        'RFCDATE': '2022-01-01'::DATE,
        'RFCTIME': '12:34:56'::TIME,
        'RFCFLOAT': 1.23456789,
        'RFCINT1': 1,
        'RFCINT2': 130,
        'RFCINT4': 32767,
        'RFCHEX3': 'XXX',
        'RFCDATA1': repeat('X', 50),
    }
});

Running query in 'duckdb:///:memory:'

RFCFLOAT,RFCCHAR1,RFCDATE,RFCTIME,RFCINT4,RFCDATA1
1.23456789,A,2022-01-01,12:34:56,32767,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [17]:
%%sql
select RFCTABLE[1].RFCCHAR1, RFCTABLE[2].RFCCHAR2, RFCTABLE[3].RFCDATE from sap_rfc_invoke('STFC_STRUCTURE', {
    'RFCTABLE': [
        {'RFCCHAR1': 'A', 'RFCCHAR2': 'AA', 'RFCDATE': '2022-01-01'::DATE, 'RFCINT4': 1}, 
        {'RFCCHAR1': 'B', 'RFCCHAR2': 'BB', 'RFCDATE': '2022-01-02'::DATE, 'RFCINT4': 2},
        {'RFCCHAR1': 'C', 'RFCCHAR2': 'CC', 'RFCDATE': '2022-01-03'::DATE, 'RFCINT4': 3},
    ]
});

Running query in 'duckdb:///:memory:'

(RFCTABLE[1]).RFCCHAR1,(RFCTABLE[2]).RFCCHAR2,(RFCTABLE[3]).RFCDATE
A,BB,2022-01-03


In [16]:
%%sql
select RFCCHAR1 from sap_rfc_invoke('STFC_STRUCTURE', {
    'RFCTABLE': [
        {'RFCCHAR1': 'A', 'RFCCHAR2': 'AA', 'RFCDATE': '2022-01-01'::DATE}, 
        {'RFCCHAR1': 'B', 'RFCCHAR2': 'BB', 'RFCDATE': '2022-01-02'::DATE},
        {'RFCCHAR1': 'C', 'RFCCHAR2': 'CC', 'RFCDATE': '2022-01-03'::DATE},
    ]
}, path='/RFCTABLE');

Running query in 'duckdb:///:memory:'

RFCCHAR1
A
B
C
X
